In [1]:
import pandas as pd
from datetime import time
from ipyfilechooser import FileChooser
from IPython.display import display
import os


# 🎛️ Interface de sélection de fichier
fc_input = FileChooser("C:/Users/AudreyHay/Documents/Carla/FED/", select_default=True,show_only_dirs=False, title="<b>Sélectionnez le fichier CSV</b>")
display(fc_input)

# 📂 Fonction appelée après sélection du fichier
def update_input_file(chooser):
    global input_file_path
    input_file_path = chooser.selected
    print(f"\n📂 Fichier sélectionné : {input_file_path}")



FileChooser(path='C:\Users\AudreyHay\Documents\Carla\FED', filename='', title='<b>Sélectionnez le fichier CSV<…

In [2]:
# Utiliser directement la propriété .selected du sélecteur
if fc_input.selected:
    input_file_path = fc_input.selected
    print(f"📂 Fichier sélectionné : {input_file_path}")
    
    df = pd.read_csv(input_file_path)
    df.columns = df.columns.str.replace(r'\+AF8-', '_', regex=True)
    print(f"✅ Fichier chargé : {len(df)} lignes, {len(df.columns)} colonnes")
    display(df)
else:
    print("❌ Veuillez d'abord sélectionner un fichier dans l'interface ci-dessus")

📂 Fichier sélectionné : C:\Users\AudreyHay\Documents\Carla\FED\Light Tracking\APPPS1\AHAD11.59\2 mois\AHAD11.59_01_08_2025au08_08_2025.CSV
✅ Fichier chargé : 2030 lignes, 16 colonnes


,MM:DD:YYYY hh:mm:ss,Library_Version,Session_type,Device_Number,Battery_Voltage,Motor_Turns,FR,Event,Active_Poke,Left_Poke_Count,Right_Poke_Count,Pellet_Count,Block_Pellet_Count,Retrieval_Time,InterPelletInterval,Poke_Time
0,8/1/2025 17:22:16,1.16.3,Light Trk,0,4.16,NaN,1,Left,Left,1,0,0,0,NaN,NaN,0.13
1,8/1/2025 17:22:19,1.16.3,Light Trk,0,4.16,3.0,1,Pellet,Left,1,0,1,0,0.06,NaN,NaN
2,8/1/2025 17:23:32,1.16.3,Light Trk,0,4.16,NaN,1,Left,Left,2,0,1,0,NaN,NaN,0.03
3,8/1/2025 17:27:19,1.16.3,Light Trk,0,4.17,NaN,1,LeftWithPellet,Left,3,0,1,0,NaN,NaN,0.10
4,8/1/2025 17:37:08,1.16.3,Light Trk,0,4.16,4.0,1,Pellet,Left,3,0,2,0,Timed_out,889.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025,8/8/2025 11:01:27,1.16.3,Light Trk,0,3.86,NaN,1,Right,Right,603,811,612,0,NaN,NaN,0.00
2026,8/8/2025 11:01:45,1.16.3,Light Trk,0,3.86,9.0,1,Pellet,Right,603,811,613,0,1.77,96.0,NaN
2027,8/8/2025 11:03:14,1.16.3,Light Trk,0,3.86,NaN,1,Right,Left,603,812,613,0,NaN,NaN,0.07
2028,8/8/2025 11:03:20,1.16.3,Light Trk,0,3.87,NaN,1,Left,Left,604,812,613,0,NaN,NaN,0.14


In [3]:
# 📌 Colonnes à supprimer
cols_to_drop = [
    'Library_Version', 'Session_type', 'Device_Number',
    'Battery_Voltage', 'Motor_Turns', 'FR'
]   
# 🧹 Supprimer les colonnes indésirables
df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)
df

,MM:DD:YYYY hh:mm:ss,Event,Active_Poke,Left_Poke_Count,Right_Poke_Count,Pellet_Count,Block_Pellet_Count,Retrieval_Time,InterPelletInterval,Poke_Time
0,8/1/2025 17:22:16,Left,Left,1,0,0,0,NaN,NaN,0.13
1,8/1/2025 17:22:19,Pellet,Left,1,0,1,0,0.06,NaN,NaN
2,8/1/2025 17:23:32,Left,Left,2,0,1,0,NaN,NaN,0.03
3,8/1/2025 17:27:19,LeftWithPellet,Left,3,0,1,0,NaN,NaN,0.10
4,8/1/2025 17:37:08,Pellet,Left,3,0,2,0,Timed_out,889.0,NaN
...,...,...,...,...,...,...,...,...,...,...
2025,8/8/2025 11:01:27,Right,Right,603,811,612,0,NaN,NaN,0.00
2026,8/8/2025 11:01:45,Pellet,Right,603,811,613,0,1.77,96.0,NaN
2027,8/8/2025 11:03:14,Right,Left,603,812,613,0,NaN,NaN,0.07
2028,8/8/2025 11:03:20,Left,Left,604,812,613,0,NaN,NaN,0.14


In [4]:

# 🕒 Convertir la colonne datetime
df['Datetime'] = pd.to_datetime(df['MM:DD:YYYY hh:mm:ss'], errors='coerce')

# 🗓️ Extraire Date et Hour
df['Date'] = df['Datetime'].dt.date
df['Hour'] = df['Datetime'].dt.time

#df.rename(columns={'Active_Poke': 'ActivePoke'}, inplace=True)

# ☀️ Déterminer la période Day/Night
def get_period(hour):
    if pd.isnull(hour):
        return None
    if time(7, 0, 1) <= hour <= time(19, 0, 0):
        return "Day"
    else:
        return "Night"

df['Period'] = df['Hour'].apply(get_period)

# 💾 Enregistrer dans le même dossier avec _modifié
folder = os.path.dirname(input_file_path)
base = os.path.splitext(os.path.basename(input_file_path))[0]
output_file = os.path.join(folder, f"{base}_Periodic.csv")

df

,MM:DD:YYYY hh:mm:ss,Event,Active_Poke,Left_Poke_Count,Right_Poke_Count,Pellet_Count,Block_Pellet_Count,Retrieval_Time,InterPelletInterval,Poke_Time,Datetime,Date,Hour,Period
0,8/1/2025 17:22:16,Left,Left,1,0,0,0,NaN,NaN,0.13,2025-08-01 17:22:16,2025-08-01,17:22:16,Day
1,8/1/2025 17:22:19,Pellet,Left,1,0,1,0,0.06,NaN,NaN,2025-08-01 17:22:19,2025-08-01,17:22:19,Day
2,8/1/2025 17:23:32,Left,Left,2,0,1,0,NaN,NaN,0.03,2025-08-01 17:23:32,2025-08-01,17:23:32,Day
3,8/1/2025 17:27:19,LeftWithPellet,Left,3,0,1,0,NaN,NaN,0.10,2025-08-01 17:27:19,2025-08-01,17:27:19,Day
4,8/1/2025 17:37:08,Pellet,Left,3,0,2,0,Timed_out,889.0,NaN,2025-08-01 17:37:08,2025-08-01,17:37:08,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025,8/8/2025 11:01:27,Right,Right,603,811,612,0,NaN,NaN,0.00,2025-08-08 11:01:27,2025-08-08,11:01:27,Day
2026,8/8/2025 11:01:45,Pellet,Right,603,811,613,0,1.77,96.0,NaN,2025-08-08 11:01:45,2025-08-08,11:01:45,Day
2027,8/8/2025 11:03:14,Right,Left,603,812,613,0,NaN,NaN,0.07,2025-08-08 11:03:14,2025-08-08,11:03:14,Day
2028,8/8/2025 11:03:20,Left,Left,604,812,613,0,NaN,NaN,0.14,2025-08-08 11:03:20,2025-08-08,11:03:20,Day


In [5]:
df.to_csv(output_file, index=False)

# ✅ Confirmation + aperçu
print("\n✅ Le fichier a bien été enregistré !")
print(f"📍 Chemin : {output_file}")
print("\n🔍 Aperçu des premières lignes du fichier modifié :")
display(df.head())



✅ Le fichier a bien été enregistré !
📍 Chemin : C:\Users\AudreyHay\Documents\Carla\FED\Light Tracking\APPPS1\AHAD11.59\2 mois\AHAD11.59_01_08_2025au08_08_2025_Periodic.csv

🔍 Aperçu des premières lignes du fichier modifié :


,MM:DD:YYYY hh:mm:ss,Event,Active_Poke,Left_Poke_Count,Right_Poke_Count,Pellet_Count,Block_Pellet_Count,Retrieval_Time,InterPelletInterval,Poke_Time,Datetime,Date,Hour,Period
0,8/1/2025 17:22:16,Left,Left,1,0,0,0,NaN,NaN,0.13,2025-08-01 17:22:16,2025-08-01,17:22:16,Day
1,8/1/2025 17:22:19,Pellet,Left,1,0,1,0,0.06,NaN,NaN,2025-08-01 17:22:19,2025-08-01,17:22:19,Day
2,8/1/2025 17:23:32,Left,Left,2,0,1,0,NaN,NaN,0.03,2025-08-01 17:23:32,2025-08-01,17:23:32,Day
3,8/1/2025 17:27:19,LeftWithPellet,Left,3,0,1,0,NaN,NaN,0.10,2025-08-01 17:27:19,2025-08-01,17:27:19,Day
4,8/1/2025 17:37:08,Pellet,Left,3,0,2,0,Timed_out,889.0,NaN,2025-08-01 17:37:08,2025-08-01,17:37:08,Day


In [6]:
# Afficher les événements uniques pour vérification
print("Types d'événements dans les données :")
print(df['Event'].unique())

# Assurer format cohérent
df['Datetime'] = pd.to_datetime(df['MM:DD:YYYY hh:mm:ss'], errors='coerce')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.date
df['Hour'] = pd.to_datetime(df['Hour'], format='%H:%M:%S', errors='coerce').dt.time

# Trier les données par date et heure
df = df.sort_values(['Date', 'Hour']).reset_index(drop=True)

# Créer un identifiant de bloc pour les périodes consécutives
# Cela va identifier chaque changement de période (Day->Night ou Night->Day)
df['Period_Block'] = (df['Period'] != df['Period'].shift(1)).cumsum()

# Définir les catégories d'événements
specific_events = [
    'LeftDuringDispense',
    'RightDuringDispense',
    'LeftWithPellet',
    'RightWithPellet',
    'RightinTimeOut',
    'LeftinTimeOut'
]

results = []

# Regrouper par bloc de période (périodes consécutives)
for (period_block), group in df.groupby('Period_Block'):
    # Le period_type est le même pour tout le groupe (Day ou Night)
    period_type = group['Period'].iloc[0]
    
    left_pokes = group[group['Event'] == 'Left'].shape[0]
    right_pokes = group[group['Event'] == 'Right'].shape[0]
    total_pokes = left_pokes + right_pokes
    correct = (group['Event'] == group['Active_Poke']).sum()
    
    # Comptage des pellets
    pellets = group[group['Event'] == 'Pellet'].shape[0]
    
    # First poke après Pellet
    pellet_indices = group[group['Event'] == 'Pellet'].index
    first_pokes = []
    for idx in pellet_indices:
        if idx + 1 in group.index:
            first_pokes.append(group.loc[idx + 1])
    first_pokes_df = pd.DataFrame(first_pokes)
    first_poke_total = len(first_pokes)
    first_poke_correct = (first_pokes_df['Event'] == first_pokes_df['Active_Poke']).sum() if not first_pokes_df.empty else 0
    
    # Calcul des pourcentages
    pct_first_poke_correct = round((first_poke_correct / first_poke_total) * 100, 2) if first_poke_total > 0 else None
    pct_correct = round((correct / total_pokes) * 100, 2) if total_pokes > 0 else None
    
    # Specific events
    spec_events = group[group['Event'].isin(specific_events)].shape[0]
    
    # Dates de début et fin du bloc
    start_date = group['Date'].min()
    end_date = group['Date'].max()
    
    results.append({
        'Period Block': period_block,
        'Period Type': period_type,
        'Start Date': start_date,
        'End Date': end_date,
        'Pellet': pellets,
        'Pokes': total_pokes,
        'Left Pokes': left_pokes,
        'Right Pokes': right_pokes,
        'Correct': correct,
        '% Correct': pct_correct,
        'First Poke': first_poke_total,
        'First Poke Correct': first_poke_correct,
        '% First Poke Correct': pct_first_poke_correct,
        'Specific Events': spec_events,
    })

# Résumé
summary_df = pd.DataFrame(results)
summary_df

Types d'événements dans les données :
['Left' 'Pellet' 'LeftWithPellet' 'Right' 'LeftDuringDispense'
 'RightDuringDispense' 'LeftinTimeOut' 'RightWithPellet']


,Period Block,Period Type,Start Date,End Date,Pellet,Pokes,Left Pokes,Right Pokes,Correct,% Correct,First Poke,First Poke Correct,% First Poke Correct,Specific Events
0,1,Day,2025-08-01,2025-08-01,3,4,3,1,3,75.00,2,1,50.00,1
1,2,Night,2025-08-01,2025-08-02,31,76,41,35,31,40.79,30,10,33.33,12
2,3,Day,2025-08-02,2025-08-02,32,72,31,41,32,44.44,32,18,56.25,4
3,4,Night,2025-08-02,2025-08-03,76,212,93,119,76,35.85,75,29,38.67,8
4,5,Day,2025-08-03,2025-08-03,19,46,23,23,19,41.30,19,8,42.11,2
5,6,Night,2025-08-03,2025-08-04,59,138,34,104,59,42.75,59,31,52.54,3
6,7,Day,2025-08-04,2025-08-04,31,63,29,34,31,49.21,30,17,56.67,1
7,8,Night,2025-08-04,2025-08-05,73,114,53,61,73,64.04,72,44,61.11,4
8,9,Day,2025-08-05,2025-08-05,33,66,22,44,33,50.00,32,17,53.12,23
9,10,Night,2025-08-05,2025-08-06,61,93,48,45,61,65.59,60,42,70.00,8


In [7]:
# Sauvegarde CSV
base = os.path.splitext(os.path.basename(output_file))[0]
output_summary = os.path.join(folder, f"{base}_global.csv")
summary_df.to_csv(output_summary, index=False)

print(f"\n✅ Fichier résumé sauvegardé avec succès !")
print(f"📍 Chemin : {output_summary}")
display(summary_df)


✅ Fichier résumé sauvegardé avec succès !
📍 Chemin : C:\Users\AudreyHay\Documents\Carla\FED\Light Tracking\APPPS1\AHAD11.59\2 mois\AHAD11.59_01_08_2025au08_08_2025_Periodic_global.csv


,Period Block,Period Type,Start Date,End Date,Pellet,Pokes,Left Pokes,Right Pokes,Correct,% Correct,First Poke,First Poke Correct,% First Poke Correct,Specific Events
0,1,Day,2025-08-01,2025-08-01,3,4,3,1,3,75.00,2,1,50.00,1
1,2,Night,2025-08-01,2025-08-02,31,76,41,35,31,40.79,30,10,33.33,12
2,3,Day,2025-08-02,2025-08-02,32,72,31,41,32,44.44,32,18,56.25,4
3,4,Night,2025-08-02,2025-08-03,76,212,93,119,76,35.85,75,29,38.67,8
4,5,Day,2025-08-03,2025-08-03,19,46,23,23,19,41.30,19,8,42.11,2
5,6,Night,2025-08-03,2025-08-04,59,138,34,104,59,42.75,59,31,52.54,3
6,7,Day,2025-08-04,2025-08-04,31,63,29,34,31,49.21,30,17,56.67,1
7,8,Night,2025-08-04,2025-08-05,73,114,53,61,73,64.04,72,44,61.11,4
8,9,Day,2025-08-05,2025-08-05,33,66,22,44,33,50.00,32,17,53.12,23
9,10,Night,2025-08-05,2025-08-06,61,93,48,45,61,65.59,60,42,70.00,8
